# Task 1: Visual Saliency Detection

In [ ]:
pip install opencv-contrib-python


Saliency Detection on an Image

In [ ]:
import cv2
import numpy as np

# Load the input image
image = cv2.imread('input.jpg')
if image is None:
    raise FileNotFoundError("Image not found.")

# Initialize the saliency detector (Spectral Residual method)
saliency = cv2.saliency.StaticSaliencySpectralResidual_create()

# Compute the saliency map
(success, saliency_map) = saliency.computeSaliency(image)
if not success:
    raise Exception("Saliency computation failed.")

# Convert the saliency map to a binary map
saliency_map = (saliency_map * 255).astype("uint8")
_, binary_map = cv2.threshold(saliency_map, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# Display the results
cv2.imshow("Original Image", image)
cv2.imshow("Saliency Map", saliency_map)
cv2.imshow("Binary Map", binary_map)
cv2.waitKey(0)
cv2.destroyAllWindows()


Saliency Detection on a Video Stream

In [ ]:
import cv2
import numpy as np

# Open the video file or capture device
cap = cv2.VideoCapture('input_video.mp4')  # Replace with 0 for webcam

# Initialize the saliency detector
saliency = cv2.saliency.MotionSaliencyBinWangApr2014_create()
saliency.setImagesize(int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
saliency.init()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale as required by the saliency detector
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute the saliency map
    success, saliency_map = saliency.computeSaliency(gray)
    if not success:
        continue

    # Convert the saliency map to a displayable format
    saliency_display = (saliency_map * 255).astype("uint8")

    # Display the results
    cv2.imshow("Original Frame", frame)
    cv2.imshow("Saliency Map", saliency_display)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


#Task 2: Unsupervised Image Segmentation using K-Means Clustering

In [ ]:
pip install opencv-python numpy


In [ ]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('landscape.jpg')
if image is None:
    raise FileNotFoundError("Image not found.")
Z = image.reshape((-1, 3))  # Reshape to a 2D array of pixels
Z = np.float32(Z)  # Convert to float32

# Define criteria and apply KMeans
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 4  # Number of clusters
ret, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

# Convert back to uint8 and reshape to original image
center = np.uint8(center)
res = center[label.flatten()]
segmented_image = res.reshape((image.shape))

# Display the results
cv2.imshow('Original Image', image)
cv2.imshow('Segmented Image', segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


#Task 3: Graph Cut Segmentation using GrabCut

In [ ]:
pip install opencv-python


In [ ]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('input.jpg')
if image is None:
    raise FileNotFoundError("Image not found.")
mask = np.zeros(image.shape[:2], np.uint8)

# Define the background and foreground models
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

# Define the rectangle containing the foreground object
rect = (50, 50, image.shape[1]-100, image.shape[0]-100)  # Adjust as needed

# Apply GrabCut algorithm
cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

# Create mask where sure and likely foreground are set to 1, others 0
mask2 = np.where((mask==2)|(mask==0), 0, 1).astype('uint8')
result = image * mask2[:, :, np.newaxis]

# Display the results
cv2.imshow('Original Image', image)
cv2.imshow('Segmented Image', result)
cv2.waitKey(0)
cv2.destroyAllWindows()
